# Xử lý file đầu vào

In [1]:
# import các thư viện
import librosa
import librosa.display
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
from pydub.playback import play 
import re
import os
from pydub import AudioSegment
from pydub.utils import which
AudioSegment.converter = which("ffmpeg")

In [2]:
# xoá dấu trong nhãn tiếng việt
def no_accent_vietnamese(s):
    s = s.lower()
    s = re.sub('[áàảãạăắằẳẵặâấầẩẫậ]', 'a', s)
    s = re.sub('[ÁÀẢÃẠĂẮẰẲẴẶÂẤẦẨẪẬ]', 'A', s)
    s = re.sub('[éèẻẽẹêếềểễệ]', 'e', s)
    s = re.sub('[ÉÈẺẼẸÊẾỀỂỄỆ]', 'E', s)
    s = re.sub('[óòỏõọôốồổỗộơớờởỡợ]', 'o', s)
    s = re.sub('[ÓÒỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢ]', 'O', s)
    s = re.sub('[íìỉĩị]', 'i', s)
    s = re.sub('[ÍÌỈĨỊ]', 'I', s)
    s = re.sub('[úùủũụưứừửữự]', 'u', s)
    s = re.sub('[ÚÙỦŨỤƯỨỪỬỮỰ]', 'U', s)
    s = re.sub('[ýỳỷỹỵ]', 'y', s)
    s = re.sub('[ÝỲỶỸỴ]', 'Y', s)
    s = re.sub('đ', 'd', s)
    s = re.sub('Đ', 'D', s)
    return s

In [8]:
# tách file từng nhãn theo giây 
def split_file(path_wav, path_txt):
    commands = list()
    wav_file = AudioSegment.from_file(path_wav)
    file = open(path_txt.format(name,n = names ), "r", encoding="utf8")
    for line in file:
        arr = line.split()
        try:
            commands.append({
                "audio" : wav_file[float(arr[0])*1000:float(arr[1])*1000] ,
                "command": no_accent_vietnamese(arr[2])
            })
        except:
            pass
    return commands

In [9]:
# Lấy file của từng người và thực hiện tách
path ="D:\\xltn\\data_raw\\{}\\"
def get_split(name):
    list_commands = []
    path_wav = [file for file in os.listdir(path.format(name)) if file.endswith(".wav")]
    path_txt = [file for file in os.listdir(path.format(name)) if file.endswith(".txt")]
    for i in range(len(path_wav)):
        list_commands = list_commands+split_file(path.format(name)+path_wav[i] ,path.format(name)+path_txt[i])
    return list_commands

In [5]:
names = ["nghia","khiem","minh","giang","huy","hoang","long"]
list_commands = []
for name in names:
    list_commands = list_commands + get_split(name)

In [6]:
list_commands = [i for i in list_commands if not (i['command'] in ["si" ,"sil"])]
len(list_commands)

6985

In [7]:
# Lưu ra một file dataset để sử dụng cho các bài tiếp theo 
list_commands_name= ["a", "b", 'nhay','ban','len','xuong','trai','phai']
for commands in list_commands_name:
    path_export = "D:\\xltn\\dataset\\{}\\file{}.wav"
    list_file = [i for i in list_commands if (i['command'] == commands)]
    for i in range(len(list_file)):
        list_file[i]["audio"].export(path_export.format(commands,i), format="wav")